# Imports and Setup

In [422]:
import sys
assert sys.version_info >= (3, 5)

import numpy as np
import pandas as pd
import statistics as stat
import random
import time

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import optproblems.cec2005 as opt
import optproblems.base as base

# Hyperparameters:

In [423]:
Number_of_Inputs = 2
Func_Domain = [-100, 100]
Stay_Within_Bounds = True
Function = opt.F1(Number_of_Inputs)

Swarm_Size = 100
Number_of_Informants = 10
Max_Number_of_Iterations = 100

Inertia = 0.5
Personal_Best = 0.5
Informants_Best = 0.5
Global_Best = 0.5
Jump_Size = 0.5
Weights = [Inertia, Personal_Best, Informants_Best, Global_Best, Jump_Size]

# Helper Functions

In [424]:
def eval(func, position):
    pos = base.Individual(position)
    func.evaluate(pos)
    return pos.objective_values

# Particle Swarm Optimisation Algorithm

### Particle Class

In [425]:
class Particle:
    def __init__(self, numInputs, domain, func, weights, boundary):
        self.position = []
        self.velocity = []
        self.bestPos = []
        self.informants: list[Particle] = []
        self.func = func
        self.domain = domain
        self.dimensions = numInputs
        self.weights = weights
        self.boundary = boundary

        for i in range(numInputs):
            self.position.append(random.uniform(domain[0], domain[1]))
            self.velocity.append(random.uniform(-np.abs(domain[1] - domain[0])/10, np.abs(domain[1] - domain[0])/10))
 
        self.result = eval(func, self.position)
        self.bestResult = self.result
        self.bestPos = self.position.copy()
    
    def evaluate(self):
        self.result = eval(self.func, self.position)

        if self.result < self.bestResult:
            self.bestResult = self.result
            self.bestPos = self.position.copy()

    def update_velocity(self, globalBestPos):
        bestInformant = self.informants[0]
        for particle in self.informants:
            if particle.bestResult < bestInformant.bestResult:
                bestInformant = particle

        for i in range(self.dimensions):
            cognitive = random.uniform(0, self.weights[1])*(self.bestPos[i] - self.position[i])
            social = random.uniform(0, self.weights[2])*(bestInformant.bestPos[i] - self.position[i])
            globalv = random.uniform(0, self.weights[3])*(globalBestPos[i] - self.position[i])
            self.velocity[i] = self.weights[0]*self.velocity[i] + cognitive + social + globalv
    
    def update_position(self):
        for i in range(self.dimensions):
            self.position[i] += self.weights[4] * self.velocity[i]
            
            if self.boundary:
                if self.position[i] < self.domain[0]:
                    self.position[i] = self.domain[0]
                elif self.position[i] > self.domain[1]:
                    self.position[i] = self.domain[1]

### Particle Swarm Optimization Algorithm Class

In [426]:
class PSO():
    def __init__(self, numInputs, domain, func, weights, swarmSize, numInformants, maxIter, boundary = True, result = True):
        self.swarm: list[Particle] = []
        self.swarmSize = swarmSize
        self.maxIter = maxIter
        self.domain = domain
        self.func = func
        self.numInputs = numInputs
        self.initSwarm = []

        for i in range(swarmSize):
            particle = Particle(numInputs, domain, func, weights, boundary)
            self.swarm.append(particle)
            self.initSwarm.append(particle.position.copy())
        
        for i in range(swarmSize):
            while len(self.swarm[i].informants) < numInformants:
                inform = random.choice(self.swarm)
                if self.swarm[i].informants.__contains__(inform) or self.swarm[i] == inform:
                    continue
                else:
                    self.swarm[i].informants.append(inform)

        self.globalBest = self.swarm[0]
        for j in range(self.swarmSize):
                if self.swarm[j].bestResult < self.globalBest.bestResult:
                    self.globalBest = self.swarm[j]

    def optimize(self):
        i = 0
        while i < self.maxIter:
            for j in range(self.swarmSize):
                self.swarm[j].evaluate()
            
            for j in range(self.swarmSize):
                if self.swarm[j].bestResult < self.globalBest.bestResult:
                    self.globalBest = self.swarm[j]

            for j in range(self.swarmSize):
                self.swarm[j].update_velocity(self.globalBest.bestPos)
                self.swarm[j].update_position()

            i+=1

    def print_result(self):
        print("Solution:")
        print(self.globalBest.bestPos)
        print("Result:")
        print(eval(self.func, self.globalBest.bestPos), self.globalBest.bestResult)

    def display_graph(self):
        if self.numInputs != 2:
            print("Display is only possible with 2 dimensions/number of inputs!")
            return
        elif type(self.domain[0]) != int or type(self.domain[1]) != int:
            print("Can't display non integer bounds!")
            return

        boundT = 2*self.domain[1] 
        x, y = np.array(np.meshgrid(np.linspace(-self.domain[1],self.domain[1],boundT), np.linspace(-self.domain[1],self.domain[1],boundT)))
        z = np.zeros([boundT, boundT])

        for i in range(boundT):
            for j in range(boundT):
                plotSoln = base.Individual(np.array([x[i][j], y[i][j]]))
                self.func.evaluate(plotSoln)
                z[i][j] = plotSoln.objective_values

        xFirst = np.zeros(self.swarmSize)                                          
        yFirst = np.zeros(self.swarmSize)
        xFinal = np.zeros(self.swarmSize) 
        yFinal = np.zeros(self.swarmSize)

        for i in range(self.swarmSize):
            xFirst[i] = self.initSwarm[i][0]
            yFirst[i] = self.initSwarm[i][1]

        for i in range(self.swarmSize):
            xFinal[i] = self.swarm[i].position[0]
            yFinal[i] = self.swarm[i].position[1]

        x_min = x.reshape(-1)[z.argmin()]
        y_min = y.reshape(-1)[z.argmin()]
 
        plt.figure(figsize=(6,4))
        plt.imshow(z, extent=[-self.domain[1], self.domain[1], -self.domain[1], self.domain[1]], origin='lower', cmap='PuBuGn', alpha=1)
        plt.colorbar()
        plt.plot([x_min], [y_min], marker='x', markersize=10, color="black", label="Optimal")
        plt.plot(self.globalBest.bestPos[0], self.globalBest.bestPos[1], marker='^', markersize=5, color="black", label="Best")
        contours = plt.contour(x, y, z, colors='black', alpha=0.4)
        plt.clabel(contours, inline=True, fontsize=8, fmt="%.0f")
        plt.scatter(xFirst, yFirst, 5, c="orange", label="first generation") 
        plt.scatter(xFinal, yFinal, 5, c="red", label="last generation")
        plt.legend()
        plt.show()


# Runnning and Testing

In [431]:
pso = PSO(Number_of_Inputs, Func_Domain, Function, Weights, Swarm_Size, Number_of_Informants, Max_Number_of_Iterations)
pso.optimize()
pso.print_result()
optSol = Function.get_optimal_solutions()[0]
Function.evaluate(optSol)
print(optSol.phenome)
pso.display_graph()

Solution:
[-0.1367000000000009, 0.1186000000000006]
Result:
90.0 90.0
[-0.1367, 0.1186]
Can't display non integer bounds!


# Testing with Performance Metrics

In [428]:
def run(runs): 
    optSol = Function.get_optimal_solutions()[0]
    Function.evaluate(optSol)

    bestParticleList : list[Particle] = []
    for i in range(runs):
      pso = PSO(Number_of_Inputs, Func_Domain, Function, Weights, Swarm_Size, Number_of_Informants, Max_Number_of_Iterations)
      pso.optimize()
      bestParticleList.append(pso.globalBest)

    avgSum = 0
    blob = []
    for i in range(runs):
      blob.append(bestParticleList[i].bestResult)
      avgSum += bestParticleList[i].bestResult
    averageObjVal = sum(blob)/len(blob)
    print("Actual =", optSol.objective_values)
    print("Average =", averageObjVal)
    accuracy = abs((optSol.objective_values - averageObjVal)/optSol.objective_values) * 100
    print("percenatge error of value = ", accuracy)
    
    stdDevObjVal = stat.pstdev(blob)
    print("The Standard Deviation of best value of the past 10 runs is ", stdDevObjVal)

    stdSum = []
    stdAvg = np.zeros(Number_of_Inputs)
    stdAcc = []
    resStdDev = []
    for i in range(Number_of_Inputs):
      stdSum = []
      for j in range(runs):
        stdSum.append(bestParticleList[j].bestPos[i])
      stdAvg[i] = sum(stdSum)/len(stdSum)
      stdAcc.append(abs((optSol.phenome[i] - stdAvg[i])/optSol.phenome[i]) * 100)
      resStdDev.append(stat.pstdev(stdSum))
    
    perErr = sum(stdAcc)/len(stdAcc)             
    print("Avg percentage error for elements of solution is ", perErr)
    
    avgStdDev = sum(resStdDev)/len(resStdDev)
    print("Avg standard deviation for elements of solution is ", avgStdDev)

In [429]:
Number_of_Inputs = 2
Func_Domain = [-0.5, 0.5]
Stay_Within_Bounds = True
Function = opt.F11(Number_of_Inputs)

Swarm_Size = 100
Number_of_Informants = 10
Max_Number_of_Iterations = 100

Inertia = 0.5
Personal_Best = 0.5
Informants_Best = 0.5
Global_Best = 0.5
Jump_Size = 1
Weights = [Inertia, Personal_Best, Informants_Best, Global_Best, Jump_Size]

In [430]:
run(10)

Actual = 90.0
Average = 90.0
percenatge error of value =  0.0
The Standard Deviation of best value of the past 10 runs is  1.7053025658242404e-14
Avg percentage error for elements of solution is  147.31775608535395
Avg standard deviation for elements of solution is  0.18847463522718966
